In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.cbook import boxplot_stats
import numpy as np
from matplotlib import cm
import geopandas
import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML

In [2]:
sns.set()

In [3]:
complaints=pd.read_csv('ppd_complaints.csv')

In [4]:
complaint_date=complaints.reset_index().set_index('complaint_id')['date_received'].to_dict()

In [5]:
po_districts=geopandas.read_file("Boundaries_District.geojson")
po_districts.head(5)

,OBJECTID,AREA,PERIMETER,DISTRICT_,DISTRICT_ID,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,geometry
0,321,None,81903.641825,1,None,1,None,01,24th St. & Wolf St.,686-3010,SPD,2.163501e+08,"POLYGON ((-75.19724 39.92944, -75.19693 39.929..."
1,322,None,63587.369399,2,None,2,None,02,Harbison Ave. & Levick St.,686-3020,NEPD,1.923461e+08,"POLYGON ((-75.05444 40.04454, -75.05482 40.044..."
2,323,None,55305.496227,3,None,3,None,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,"POLYGON ((-75.13205 39.89932, -75.13470 39.894..."
3,324,None,71919.797243,5,None,5,None,05,Ridge Ave. & Cinnaminson St.,686-3050,NWPD,2.130442e+08,"POLYGON ((-75.20430 40.03468, -75.20403 40.034..."
4,325,None,34655.320856,6,None,6,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,"POLYGON ((-75.13437 39.95294, -75.13524 39.950..."


In [6]:
disciplines=pd.read_csv('ppd_complaint_disciplines.csv')

In [7]:
disciplines['date_recieved']=disciplines.complaint_id.map(complaint_date)

In [8]:
disciplines['district_num']=pd.to_numeric(disciplines.po_assigned_unit.dropna().apply(lambda x: x.split(' ')[0][:2]),errors='coerce')

In [ ]:
inc=geopandas.read_file("../incidents_part1_part2.geojson")

In [ ]:
complaintsbyofficer=disciplines.groupby('officer_id').complaint_id.count().sort_values(ascending=False)

In [ ]:
complaintsbyofficer.describe().to_frame().style.format("{:.2f}")

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(20,10))
complaintsbyofficer.hist(figsize=(20,10));
ax.tick_params(labelsize=24)

In [ ]:
unit_officer=disciplines.groupby(['po_assigned_unit','officer_id']).complaint_id.count().reset_index()

In [ ]:
disciplines[disciplines['officer_id']==29180642].head(10)

In [ ]:
unit_officer=disciplines.groupby(['district_num','officer_id']).complaint_id.count().reset_index()

In [ ]:
display(HTML('<h1>TOP 10 OFFICERS BY NUMBER OF COMPLAINTS</h1>'))
unit_officer[unit_officer['district_num'].isin(po_districts.DISTRICT_.unique())].sort_values('complaint_id',ascending=False).head(10).style.hide_index()

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(35,10))
g=sns.boxplot(x="district_num", y="complaint_id",data=unit_officer[unit_officer['district_num'].isin(po_districts.DISTRICT_.unique())], ax=ax)
plt.xticks(rotation=90);
ax.tick_params(labelsize=24);
fig.suptitle('District Complaints/ Officer', fontsize=32);
ax.set_xlabel('District Number', fontsize=24);
ax.set_ylabel('Number of Complaints', fontsize=24);

In [ ]:
outlier_officers=[]
for c, g in unit_officer.reset_index().groupby('district_num'):
    stats=boxplot_stats(g.complaint_id)
    outliers = [y for stat in boxplot_stats(g['complaint_id']) for y in stat['fliers']]
    outlier_officers=np.append(outlier_officers,list(g[g['complaint_id'].isin(outliers)].officer_id.values))

In [ ]:
outlier_officers

In [ ]:
outlier_race=disciplines[disciplines['officer_id'].isin(outlier_officers)].groupby('officer_id')['po_race'].unique()

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(20,10))
outlier_race.apply(lambda x:x[0]).value_counts().plot(kind='bar', ax=ax);
ax.tick_params(labelsize=24)

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(20,10))
disciplines.groupby('officer_id').po_race.unique().apply(lambda x:x[0]).value_counts().plot(kind='bar');
ax.tick_params(labelsize=24)

In [ ]:
colors_tc=cm.Spectral(np.linspace(0, 1,len(disciplines.investigative_findings.unique()) ) )

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(10,10))
disciplines.investigative_findings.value_counts().plot(kind='pie', ax=ax,textprops={'fontsize': 14},colors=colors_tc,wedgeprops={"edgecolor":"k",'linewidth': 1, 'antialiased': True});

In [ ]:
race_results=disciplines.groupby('po_race').investigative_findings.value_counts().to_frame()
race_results.columns=['counts']
race_results=race_results.reset_index()
r_f=race_results[~race_results.investigative_findings.isin(['Pending'])]

In [ ]:

cdict={}
for i,c in zip(disciplines.investigative_findings.unique(), colors_tc):
    cdict[i]=c

def plt_pie(x,y,**kwargs):
    cd=[cdict[l] for l in x] 
    plt.pie(y,labels=x, colors=cd,wedgeprops={"edgecolor":"k",'linewidth': 1, 'antialiased': True})

In [ ]:
g=sns.FacetGrid(r_f,col="po_race", col_wrap=2, sharey=False,height=6)
g=(g.map(plt_pie, "investigative_findings", "counts",cdict=cdict))

In [ ]:
pd.pivot_table(r_f,values='counts',index=['po_race'],columns=['investigative_findings']).apply(lambda x: x / x.sum(), axis=1).fillna(0).style.format("{:.2%}")

In [ ]:
ps=geopandas.read_file("Police_Stations.geojson")

In [ ]:
comp_dict=disciplines.groupby('district_num').complaint_id.count().to_dict()

In [ ]:
po_districts['comp_count']=pd.to_numeric(po_districts.DISTRICT_).apply(lambda x: comp_dict[x])

fig, ax = plt.subplots(1, 1,figsize=(30,30))
po_districts.to_crs(epsg=3857).plot(column='comp_count',cmap='YlOrRd',legend=True, ax=ax, alpha=0.5)
po_districts.to_crs(epsg=3857).boundary.plot(ax=ax, color='k')
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)

for l,name in zip(po_districts.to_crs(epsg=3857).centroid,po_districts.DISTRICT_):
    plt.text(l.x,l.y,name,ha='center', va='center',fontsize='32')
    
ax.set_axis_off()
ax.set_title('POLICE DISTRICT, COMPLAINT COUNTS',fontsize='56');

In [ ]:
outliers_dict=disciplines[disciplines['officer_id'].isin(outlier_officers)].groupby('district_num').complaint_id.count().to_dict()

In [ ]:
po_districts['outlier_count']=pd.to_numeric(po_districts.DISTRICT_).apply(lambda x: outliers_dict[x])

fig, ax = plt.subplots(1, 1,figsize=(30,30))
po_districts.to_crs(epsg=3857).plot(column='outlier_count',cmap='YlOrRd',legend=True, ax=ax, alpha=0.5)
po_districts.to_crs(epsg=3857).boundary.plot(ax=ax, color='k')
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)
for l,name in zip(po_districts.to_crs(epsg=3857).centroid,po_districts.DISTRICT_):
    plt.text(l.x,l.y,name,ha='center', va='center',fontsize='32')
ax.set_axis_off()
ax.set_title('POLICE DISTRICT, COMPLAINTS BY OUTLIER OFFICER',fontsize='56');

In [ ]:
inc_in_z=geopandas.sjoin(inc,po_districts,how='inner',op='within')

In [ ]:
inc_in_d=inc_in_z.groupby('DISTRICT_').objectid.count()

In [ ]:
inc_in_d_dict=inc_in_d.to_dict()

In [ ]:
po_districts['incidents']=po_districts['DISTRICT_'].apply(lambda x: inc_in_d_dict[x])

In [ ]:
po_districts['ratio']=po_districts.apply(lambda x:(x.comp_count*1000)/x.incidents,axis=1)

In [ ]:
ratio_dict=po_districts[['DISTRICT_','ratio']].set_index('DISTRICT_').to_dict()['ratio']

In [ ]:
inc_in_d=inc_in_d.loc[lambda x: x.index != 77]

In [ ]:
xmax=inc_in_d.max()
xmin=inc_in_d.min()

def scale_into_range(x,a,b,xmin,xmax):
    res=(((b-a)*(x-xmin))/(xmax-xmin))+a
    return res

inc_in_d_scale=inc_in_d.apply(lambda x: scale_into_range(x,1000,20000,xmin,xmax))
inc_in_d_scale_dict=inc_in_d_scale.to_dict()



In [ ]:
po_districts_n=po_districts[po_districts['DISTRICT_']!=77].copy()
po_districts_n['Inc_Sc']=po_districts_n.DISTRICT_.map(inc_in_d_scale)

In [ ]:
po_districts_n['comp_count']=pd.to_numeric(po_districts_n.DISTRICT_).apply(lambda x: comp_dict[x])

fig, ax = plt.subplots(1, 1,figsize=(30,30))
po_districts_n.to_crs(epsg=3857).plot(column='comp_count',cmap='Blues', ax=ax, alpha=0.5)

po_districts_n.to_crs(epsg=3857).boundary.plot(ax=ax, color='k')
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)
plt.scatter(list(po_districts_n.to_crs(epsg=3857).centroid.x),list(po_districts_n.to_crs(epsg=3857).centroid.y),alpha=0.5,s=list(po_districts_n.Inc_Sc),c=list(po_districts_n.ratio),cmap='YlOrRd',edgecolor='k')  
for l,name in zip(po_districts_n.to_crs(epsg=3857).centroid,po_districts_n.DISTRICT_):
    mlab='District: '+str(name)+'\nIncidents: '+'{:,}'.format(inc_in_d_dict[name])+'\nComplaints: '+str(comp_dict[name])
    plt.text(l.x,l.y,mlab,ha='center', va='center',fontsize='12')

axins = inset_axes(ax,
                   width="5%",  # width = 5% of parent_bbox width
                   height="100%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(-0.1, 0., 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0,
                   )

sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=po_districts_n.comp_count.min(), vmax=po_districts_n.comp_count.max()))
cbr = fig.colorbar(sm, cax=axins,alpha=0.5)
cbr.ax.tick_params(labelsize=18) 
cbr.set_label(label='Complaint Counts',fontsize=18)

axins = inset_axes(ax,
                   width="5%",  # width = 5% of parent_bbox width
                   height="100%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(1.1, 0., 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0,
                   )

sm2 = plt.cm.ScalarMappable(cmap='YlOrRd', norm=plt.Normalize(vmin=po_districts_n.ratio.min(), vmax=po_districts_n.ratio.max()))
cbr2 = fig.colorbar(sm2, cax=axins,alpha=0.5)
cbr2.ax.tick_params(labelsize=18) 
cbr2.set_label(label='Complaints per 1000 Incidents',fontsize=18)

plt.text(0.8, 0.1, 'Marker Size related to Number of Incidents\n District 77 (Airport) not included',fontsize=26, horizontalalignment='center',verticalalignment='center', transform=ax.transAxes,bbox={'ec':'k','fc':'w'})
ax.set_axis_off()
ax.set_title('POLICE DISTRICT, RATIO OF COMPLAINTS PER 1000 INCIDENTS',fontsize='56')

In [ ]:
dn_results=disciplines.groupby('district_num').investigative_findings.value_counts().to_frame()
dn_results.columns=['counts']
dn_results=dn_results.reset_index()
dn_f=dn_results[~dn_results.investigative_findings.isin(['Pending'])]

In [ ]:
display(HTML('<h1>INVESTIGATION RESULTS PER DISTRICT</h1>'))
dn_perc=pd.pivot_table(dn_f,values='counts',index=['district_num'],columns=['investigative_findings']).apply(lambda x: x / x.sum(), axis=1)
dn_perc.loc[po_districts.DISTRICT_.unique()].sort_values('Sustained Finding',ascending=False).style.format("{:.2%}")

In [ ]:
display(HTML('<h1>SUSTAINED FINDINGS STATISTICS</h1>'))
dn_perc.loc[po_districts.DISTRICT_.unique()]['Sustained Finding'].describe().to_frame().loc[lambda x:x.index!='count'].style.format("{:.2%}")

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(20,10))
dn_perc.loc[po_districts.DISTRICT_.unique()].plot(kind='bar',figsize=(20,10),stacked=True,ax=ax);
ax.set_title('COMPLAINT INVESTIGATION RESULTS PER DISTRICT',fontsize='32');
ax.tick_params(labelsize=18)

In [ ]:
c_blocks=geopandas.read_file("../Census_Blocks_2010.geojson")

cblocks_in_pd=geopandas.sjoin(c_blocks,po_districts,how='inner',op='intersects')

In [ ]:
state_codes=requests.get('https://api.census.gov/data/2010/dec/sf1?get=NAME&for=state:*')

In [ ]:
state_dict={}
for x in state_codes.json()[1:]:
    state_dict[x[0]]=x[1]
state_code=state_dict['Pennsylvania']

In [ ]:
county_codes=requests.get('https://api.census.gov/data/2010/dec/sf1?get=NAME&for=county:*&in=state:'+state_code)

In [ ]:
county_dict={}
for x in county_codes.json()[1:]:
    county_dict[x[0]]=x[2]
county_code=county_dict['Philadelphia County, Pennsylvania']

In [ ]:
census_pop=requests.get('https://api.census.gov/data/2018/acs/acs5?get=B00001_001E&for=block%20group:*&in=state:'+state_code+'%20'+'county:'+county_code)

In [ ]:
c_bgroup=geopandas.read_file("Census_Block_Groups_2010.geojson")
c_bgroup['bkey']=c_bgroup.TRACTCE10+['|']+c_bgroup.BLKGRPCE10
cbgroups_in_pd=geopandas.sjoin(c_bgroup,po_districts,how='inner',op='intersects')

In [ ]:
census_pop_df=pd.DataFrame(census_pop.json()[1:],columns=census_pop.json()[0])

In [ ]:
census_race=requests.get('https://api.census.gov/data/2018/acs/acs5?get=group(B02001)&for=block%20group:*&in=state:'+state_code+'%20'+'county:'+county_code)

In [ ]:
census_race_df=pd.DataFrame(census_race.json()[1:],columns=census_race.json()[0])

In [ ]:
census_vars=requests.get('https://api.census.gov/data/2018/acs/acs5/variables.html')

In [ ]:
soup = BeautifulSoup(census_vars.content,'html.parser')

In [ ]:
data=[]
table = soup.find('table')
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [ ]:
var_dict=pd.DataFrame(data)[[0,1]].set_index(0).to_dict()[1]

In [ ]:
new_cols=[]
for x in census_race_df.columns.values:
    try: 
        val=var_dict[x]
    except:
        val=x
    new_cols.append(val)

In [ ]:
census_race_df.columns=new_cols

In [ ]:
race_cols_est=['Geography',
'Estimate!!Total',
'Estimate!!Total!!White alone',
'Estimate!!Total!!Black or African American alone',
'Estimate!!Total!!American Indian and Alaska Native alone',
'Estimate!!Total!!Asian alone',
'Estimate!!Total!!Native Hawaiian and Other Pacific Islander alone',
'Estimate!!Total!!Some other race alone',
'Estimate!!Total!!Two or more races',
'Estimate!!Total!!Two or more races!!Two races including Some other race',
'Estimate!!Total!!Two or more races!!Two races excluding Some other race, and three or more races',
'state',
'county',
'tract',
'block group']

In [ ]:
census_race_df=census_race_df[race_cols_est].copy()

In [ ]:
new_cols=[]
for x in race_cols_est:
    if 'Estimate' in x:
        if len(x.split('!!'))<=2:
            val=x.split('!!')[1]
        else:
            val='-'.join(x.split('!!')[2:])
    else:
        val=x
    new_cols.append(val)

In [ ]:
census_race_df.columns=new_cols

In [ ]:
census_race_df['bkey']=census_race_df.apply(lambda x: x['tract']+'|'+x['block group'],axis=1)

In [ ]:
census_race_df.set_index('bkey',inplace=True)

In [ ]:
census_race_df.drop(['Geography','Total','state','county','tract','block group'], axis=1);

In [ ]:
cb_div=cbgroups_in_pd.join(census_race_df.drop(['Geography','Total','state','county','tract','block group'], axis=1),on='bkey',how='left')

In [ ]:
race_col=['White alone',
'Black or African American alone',
'American Indian and Alaska Native alone', 'Asian alone',
'Native Hawaiian and Other Pacific Islander alone',
'Some other race alone', 'Two or more races', 'Two or more races-Two races including Some other race',
'Two or more races-Two races excluding Some other race, and three or more races']

In [ ]:
for c in race_col:
    cb_div[c]=cb_div[c].astype('float');

In [ ]:
display(HTML('<h1>DISTRIC RACE PERCENTAGE</h1>'))
district_race_perc=cb_div[['DISTRICT_']+race_col].groupby('DISTRICT_').sum().apply(lambda x: x / x.sum(), axis=1).dropna()
district_race_perc.style.format("{:.2%}")

In [ ]:
cats=['White alone','Black or African American alone','Asian alone']
district_race_perc['Other']=district_race_perc.drop(cats,axis=1).sum(axis=1)

In [ ]:
white_dict=district_race_perc['White alone'].to_dict()
black_dict=district_race_perc['Black or African American alone'].to_dict()
asian_dict=district_race_perc['Asian alone'].to_dict()
other_dict=district_race_perc['Other'].to_dict()

In [ ]:
po_districts_n['perc_white']=pd.to_numeric(po_districts_n.DISTRICT_).map(white_dict)
po_districts_n['perc_black']=pd.to_numeric(po_districts_n.DISTRICT_).map(black_dict)
po_districts_n['perc_asian']=pd.to_numeric(po_districts_n.DISTRICT_).map(asian_dict)
po_districts_n['perc_other']=pd.to_numeric(po_districts_n.DISTRICT_).map(other_dict)

fig, ax = plt.subplots(1, 1,figsize=(30,30))
po_districts_n.to_crs(epsg=3857).plot(column='perc_black',cmap='Greys', ax=ax, alpha=0.8)

po_districts_n.to_crs(epsg=3857).boundary.plot(ax=ax, color='k')
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)
plt.scatter(list(po_districts_n.to_crs(epsg=3857).centroid.x),list(po_districts_n.to_crs(epsg=3857).centroid.y),alpha=0.5,s=list(po_districts_n.Inc_Sc),c=list(po_districts_n.ratio),cmap='YlOrRd',edgecolor='k')  
for l,name in zip(po_districts_n.to_crs(epsg=3857).centroid,po_districts_n.DISTRICT_):
    mlab='District: '+str(name)+'\nIncidents: '+'{:,}'.format(inc_in_d_dict[name])+'\nComplaints: '+str(comp_dict[name])
    plt.text(l.x,l.y,mlab,ha='center', va='center',fontsize='12',bbox={'ec':'k','fc':'w'})

axins = inset_axes(ax,
                   width="5%",  # width = 5% of parent_bbox width
                   height="100%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(-0.1, 0., 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0,
                   )

sm = plt.cm.ScalarMappable(cmap='Greys', norm=plt.Normalize(vmin=0.0, vmax=1))
cbr = fig.colorbar(sm, cax=axins,alpha=0.5)
cbr.ax.tick_params(labelsize=18) 
cbr.set_label(label='Percentage of population identifying as "Black alone"',fontsize=18)

axins = inset_axes(ax,
                   width="5%",  # width = 5% of parent_bbox width
                   height="100%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(1.01, 0., 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0,
                   )

sm2 = plt.cm.ScalarMappable(cmap='YlOrRd', norm=plt.Normalize(vmin=po_districts_n.ratio.min(), vmax=po_districts_n.ratio.max()))
cbr2 = fig.colorbar(sm2, cax=axins,alpha=0.8)
cbr2.ax.tick_params(labelsize=18) 
cbr2.set_label(label='Complaints per 1000 Incidents',fontsize=18)

plt.text(0.8, 0.1, 'Marker Size related to Number of Incidents\n District 77 (Airport) not included',fontsize=26, horizontalalignment='center',verticalalignment='center', transform=ax.transAxes,bbox={'ec':'k','fc':'w'})
ax.set_axis_off()
ax.set_title('POLICE DISTRICT, RATIO OF COMPLAINTS PER 1000 INCIDENTS \n PERCENTAGE OF POPULATION IDENTIFYING AS BLACK',fontsize='32')
plt.savefig('PHL_MAP')

In [ ]:
format_dict = {'incidents':'{0:,.0f}', 'ratio': '{:.2f}', 'perc_other': '{:.2%}', 'perc_white': '{:.2%}','perc_black': '{:.2%}','perc_asian': '{:.2%}'}

In [ ]:
display(HTML('<h1>DISTRIC RACE PERCENTAGE, RATIO OF INCIDENTS </h1>'))

po_districts_n[['DISTRICT_','perc_white','perc_black','perc_asian','perc_other','ratio','incidents','comp_count']].set_index('DISTRICT_').sort_values('ratio',ascending=False).style.format(format_dict).highlight_max(subset=['perc_white','perc_black','perc_asian','perc_other'],axis=1).background_gradient(cmap='Reds',subset=['ratio'])

In [ ]:
po_districts_n.columns

In [ ]:
tbl_dict={'DISTRICT_':'District',
          'perc_white':'Percentage White',
          'perc_black':'Percentage Black',
          'perc_asian':'Percentage Asian',
          'perc_other':'Percentage Other',
          'ratio':'Ratio Incidents/Complaints',
         'incidents':'Outlier '}